In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd


In [4]:
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.7/345.7 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [18]:
import pandas as pd
import numpy as np
import random
from sentence_transformers import SentenceTransformer, util

In [12]:
jd_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/FYP/JD.csv")
resume_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/FYP/Resumes.csv")


In [14]:
resume_df.head()

,Resume,Category
0,database administrator family private care llc...,Database Administrator
1,database administrator database administrator ...,Database Administrator
2,oracle database administrator oracle database ...,Database Administrator
3,amazon redshift administrator and etl develope...,Database Administrator
4,scrum master scrum master scrum master richmon...,Database Administrator


In [15]:
jd_df.head()

,Job_Description,Category
0,web developer frontend web developers design a...,Web Developer
1,network administrator protect an organizations...,Network Administrator
2,systems administrator it support specialists p...,Systems Administrator
3,database administrator a database developer de...,Database Administrator
4,software developer mobile app developers desig...,Software Developer


In [16]:
# Drop any null values
resume_df = resume_df.dropna(subset=["Resume", "Category"])
jd_df = jd_df.dropna(subset=["Job_Description", "Category"])

In [19]:
model = SentenceTransformer("BAAI/bge-small-en-v1.5")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [20]:
all_pairs = []

for i, res_row in resume_df.iterrows():
    res_text = res_row["Resume"]
    res_cat = res_row["Category"]
    res_emb = model.encode(res_text, convert_to_tensor=True)

    same_cat_jds = jd_df[jd_df["Category"] == res_cat]
    if len(same_cat_jds) < 5:
        continue

    # Positive Pair
    pos_jd = same_cat_jds.sample(5).iloc[0]
    pos_emb = model.encode(pos_jd["Job_Description"], convert_to_tensor=True)
    sim = util.cos_sim(res_emb, pos_emb).item()
    pos_score = round(np.interp(sim, [0.75, 0.95], [85, 100]), 2)

    all_pairs.append({
        "Resume": res_text,
        "Job_Description": pos_jd["Job_Description"],
        "Category": res_cat,
        "Type": "positive",
        "Score": pos_score
    })

    # Semi-Negative Pair
    low_sim_jd = same_cat_jds.sample(5).iloc[1]
    low_emb = model.encode(low_sim_jd["Job_Description"], convert_to_tensor=True)
    sim_low = util.cos_sim(res_emb, low_emb).item()
    semi_score = round(np.interp(sim_low, [0.3, 0.5], [40, 60]), 2)

    all_pairs.append({
        "Resume": res_text,
        "Job_Description": low_sim_jd["Job_Description"],
        "Category": res_cat,
        "Type": "semi-negative",
        "Score": semi_score
    })

    # Negative Pair
    diff_cat_jds = jd_df[jd_df["Category"] != res_cat]
    if len(diff_cat_jds) == 0:
        continue
    neg_jd = diff_cat_jds.sample(1).iloc[0]
    neg_emb = model.encode(neg_jd["Job_Description"], convert_to_tensor=True)
    sim_neg = util.cos_sim(res_emb, neg_emb).item()
    neg_score = round(np.interp(sim_neg, [0.0, 0.3], [0, 30]), 2)

    all_pairs.append({
        "Resume": res_text,
        "Job_Description": neg_jd["Job_Description"],
        "Category": f"{res_cat}→{neg_jd['Category']}",
        "Type": "negative",
        "Score": neg_score
    })

    if i % 100 == 0:
        print(f"✅ Processed {i} resumes...")

print("🎉 All resume pairs generated!")


✅ Processed 0 resumes...
✅ Processed 100 resumes...
✅ Processed 200 resumes...
✅ Processed 300 resumes...
✅ Processed 400 resumes...
✅ Processed 500 resumes...
✅ Processed 600 resumes...
✅ Processed 700 resumes...
✅ Processed 800 resumes...
✅ Processed 900 resumes...
✅ Processed 1000 resumes...
✅ Processed 1100 resumes...
✅ Processed 1200 resumes...
✅ Processed 1300 resumes...
✅ Processed 1400 resumes...
✅ Processed 1500 resumes...
✅ Processed 1600 resumes...
✅ Processed 1700 resumes...
✅ Processed 1800 resumes...
✅ Processed 1900 resumes...
✅ Processed 2000 resumes...
✅ Processed 2100 resumes...
✅ Processed 2200 resumes...
✅ Processed 2300 resumes...
✅ Processed 2400 resumes...
✅ Processed 2500 resumes...
✅ Processed 2600 resumes...
✅ Processed 2700 resumes...
✅ Processed 2800 resumes...
✅ Processed 2900 resumes...
✅ Processed 3000 resumes...
✅ Processed 3100 resumes...
✅ Processed 3200 resumes...
✅ Processed 3300 resumes...
✅ Processed 3400 resumes...
✅ Processed 3500 resumes...
✅ Pr

In [21]:
final_df = pd.DataFrame(all_pairs)
output_path = "/content/drive/MyDrive/Colab Notebooks/FYP/resume_jd_labeled_buckets.csv"
final_df.to_csv(output_path, index=False)
print(f"✅ Labeled data saved to: {output_path}")


✅ Labeled data saved to: /content/drive/MyDrive/Colab Notebooks/FYP/resume_jd_labeled_buckets.csv


In [27]:
import pandas as pd

# Load the labeled dataset
file_path = "/content/drive/MyDrive/Colab Notebooks/FYP/resume_jd_labeled_buckets.csv"
df = pd.read_csv(file_path)

# Show basic info
print("✅ File Loaded Successfully!\n")

# Total rows
print(f"🔢 Total Rows: {len(df)}")

# Print all unique categories on new lines
print("🔠 Unique Categories:\n")
for cat in df['Category'].unique():
    print(cat)

✅ File Loaded Successfully!

🔢 Total Rows: 67158
🔠 Unique Categories:

Database Administrator
Database Administrator→Java Developer
Database Administrator→Systems Administrator
Database Administrator→Network Administrator
Database Administrator→Data Scientist
Database Administrator→Web Developer
Systems Administrator
Systems Administrator→Web Developer
Database Administrator→Software Developer
Software Developer
Software Developer→Java Developer
Web Developer
Web Developer→Systems Administrator
Software Developer→Systems Administrator
Systems Administrator→Software Developer
Systems Administrator→Java Developer
Network Administrator
Network Administrator→Java Developer
Software Developer→Data Scientist
Network Administrator→Systems Administrator
Software Developer→Network Administrator
Systems Administrator→Database Administrator
Web Developer→Network Administrator
Network Administrator→Database Administrator
Systems Administrator→Data Scientist
Java Developer
Java Developer→Database A

In [23]:
df.head()

,Resume,Job_Description,Category,Type,Score
0,database administrator family private care llc...,database administrator a database analyst exam...,Database Administrator,positive,85.00
1,database administrator family private care llc...,database administrator a database analyst exam...,Database Administrator,semi-negative,60.00
2,database administrator family private care llc...,java developer java backend developers special...,Database Administrator→Java Developer,negative,30.00
3,database administrator database administrator ...,database administrator a database developer de...,Database Administrator,positive,85.38
4,database administrator database administrator ...,database administrator a database developer de...,Database Administrator,semi-negative,60.00


In [24]:
# Find min and max of the Score column
min_score = df['Score'].min()
max_score = df['Score'].max()

print(f"🔽 Minimum Score: {min_score}")
print(f"🔼 Maximum Score: {max_score}")


🔽 Minimum Score: 30.0
🔼 Maximum Score: 91.91


🔠 Unique Categories:

Database Administrator
Database Administrator→Java Developer
Database Administrator→Systems Administrator
Database Administrator→Network Administrator
Database Administrator→Data Scientist
Database Administrator→Web Developer
Systems Administrator
Systems Administrator→Web Developer
Database Administrator→Software Developer
Software Developer
Software Developer→Java Developer
Web Developer
Web Developer→Systems Administrator
Software Developer→Systems Administrator
Systems Administrator→Software Developer
Systems Administrator→Java Developer
Network Administrator
Network Administrator→Java Developer
Software Developer→Data Scientist
Network Administrator→Systems Administrator
Software Developer→Network Administrator
Systems Administrator→Database Administrator
Web Developer→Network Administrator
Network Administrator→Database Administrator
Systems Administrator→Data Scientist
Java Developer
Java Developer→Database Administrator
Systems Administrator→Network Admini